# LightBGM 모델 

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import joblib

# 데이터 준비 및 전처리

## final_training_data_sampled_1500_ood_label.csv

In [2]:
#학습 데이터 준비
df = pd.read_csv("final_training_data_sampled_1500_ood_label.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

In [3]:
df.shape

(9000, 51)

In [4]:
df.info

<bound method DataFrame.info of           mfcc_1      mfcc_2      mfcc_3     mfcc_4     mfcc_5     mfcc_6  \
0    -436.866821  143.990936   74.952354  45.915867  33.583057  29.956610   
1    -369.858307  125.310371   72.073753  50.019867  36.092926  30.252110   
2    -453.349518  113.330528   76.104362  50.643452  42.205036  40.748249   
3    -411.235504  116.592606   75.703476  47.744938  35.118542  29.894009   
4    -395.125549  146.027084   81.780304  48.816242  33.849194  31.080847   
...          ...         ...         ...        ...        ...        ...   
8995 -170.988900  208.548900  -95.111750  31.437998 -34.982487  -3.055256   
8996 -445.657200  170.794540   38.628956  31.480406  12.929362  33.918636   
8997 -173.931100  225.113630 -121.169975  17.288454 -13.494496 -11.085254   
8998 -474.502300  172.619000   35.576990  25.952785  12.924660  32.762737   
8999 -247.981190  190.959850   -7.036764  34.192413   6.349187  12.332376   

         mfcc_7     mfcc_8     mfcc_9    mf

In [5]:
df['category_03'].value_counts()

차량사이렌     1500
이륜차주행음    1500
차량주행음     1500
이륜차경적     1500
차량경적      1500
고양이        250
발전기        250
개          250
공구         250
항타기        250
콘크리트펌프     250
Name: category_03, dtype: int64

## final_training_data_sampled_1500_category_03.csv

In [6]:
#학습 데이터 준비
df2 = pd.read_csv("final_training_data_sampled_1500_category_03.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df2 = df2[feature_cols + ['category_03']]

In [7]:
df2.shape

(16500, 51)

In [8]:
df2['category_03'].value_counts()

차량사이렌     1500
고양이       1500
공구        1500
이륜차주행음    1500
차량경적      1500
발전기       1500
개         1500
항타기       1500
차량주행음     1500
이륜차경적     1500
콘크리트펌프    1500
Name: category_03, dtype: int64

# 모델 생성

## LightGBM

## final_training_data_sampled_1500_ood_label

In [9]:
# 소음 카테고리 변환 함수
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values  # MFCC 특징 값
y = df['label'].values       # 레이블

# 레이블 인코딩 (문자 → 숫자)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터셋 분할 (훈련:테스트 = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# LightGBM 모델 정의 및 학습
model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = model.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_test, y_pred, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.9261
Classification Report:
               precision    recall  f1-score   support

        기타소음       0.95      0.95      0.95       300
       이륜차경적       0.99      0.91      0.94       300
      이륜차주행음       0.86      0.90      0.88       300
        차량경적       0.92      0.94      0.93       300
       차량사이렌       0.95      0.96      0.95       300
       차량주행음       0.90      0.90      0.90       300

    accuracy                           0.93      1800
   macro avg       0.93      0.93      0.93      1800
weighted avg       0.93      0.93      0.93      1800

Confusion Matrix:
 [[284   0   4   1  10   1]
 [  4 272   4  20   0   0]
 [  6   0 271   1   2  20]
 [  0   4  12 281   0   3]
 [  3   0   2   2 288   5]
 [  2   0  23   0   4 271]]


In [10]:
df['label'].value_counts()

차량사이렌     1500
기타소음      1500
이륜차주행음    1500
차량주행음     1500
이륜차경적     1500
차량경적      1500
Name: label, dtype: int64

### 모델 저장 - final_training_data_sampled_1500_ood_label

In [12]:
# ✅ 랜덤 포레스트 모델 저장
joblib.dump(model, 'rf_6classfication_ood_label.pkl')
print("모델이 rf_6classfication_ood_label.pkl 파일로 저장되었습니다.")

# ✅ 모델 로드 테스트
rf_model = joblib.load('rf_6classfication_ood_label.pkl')
print("모델이 성공적으로 로드되었습니다.")

모델이 rf_6classfication_ood_label.pkl 파일로 저장되었습니다.
모델이 성공적으로 로드되었습니다.


### 새로운 wav 파일 분류 함수

In [34]:
def classify_audio(audio_path):
    try:
        # 음원 파일 로드
        y, sr = librosa.load(audio_path, sr=44100)

        # MFCC 특징 추출
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)

        # MFCC 평균값 사용
        mfccs_processed = np.mean(mfccs, axis=1).reshape(1, -1)

        # 예측 수행
        predicted_label_encoded = model.predict(mfccs_processed)[0]
        predicted_label = label_encoder.inverse_transform([predicted_label_encoded])[0]

        return predicted_label

    except Exception as e:
        print(f"Error processing audio file: {e}")
        return "Unknown"

## final_training_data_sampled_1500_category_03

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 소음 카테고리 변환 함수
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df2['label'] = df2['category_03'].apply(categorize_noise)
df2 = df2.drop('category_03', axis=1)

X = df2[feature_cols].values  # MFCC 특징 값
y = df2['label'].values       # 레이블

# 레이블 인코딩 (문자 → 숫자)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터셋 분할 (훈련:테스트 = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# LightGBM 모델 정의 및 학습
model2 = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
model2.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = model2.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_test, y_pred, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9424
Classification Report:
               precision    recall  f1-score   support

        기타소음       0.95      1.00      0.97      1800
       이륜차경적       0.99      0.92      0.95       300
      이륜차주행음       0.91      0.73      0.81       300
        차량경적       0.92      0.90      0.91       300
       차량사이렌       0.98      0.94      0.96       300
       차량주행음       0.87      0.89      0.88       300

    accuracy                           0.94      3300
   macro avg       0.94      0.90      0.92      3300
weighted avg       0.94      0.94      0.94      3300

Confusion Matrix:
 [[1794    0    1    3    0    2]
 [   7  275    2   16    0    0]
 [  50    0  220    3    2   25]
 [  12    4    4  271    2    7]
 [  12    0    1    0  282    5]
 [  17    0   14    0    1  268]]


In [14]:
df2['label'].value_counts()

기타소음      9000
차량사이렌     1500
이륜차주행음    1500
차량경적      1500
차량주행음     1500
이륜차경적     1500
Name: label, dtype: int64

### 모델 저장 - final_training_data_sampled_1500_category_03

In [15]:
import joblib

# 모델 저장
joblib.dump(model2, 'rf_6classfication_category_03.pkl')
print("모델이 rf_6classfication_category_03.pkl 파일로 저장되었습니다.")

# LightGBM 모델 로드
model2 = joblib.load('rf_6classfication_category_03.pkl')
print("모델이 성공적으로 로드되었습니다.")

모델이 rf_6classfication_category_03.pkl 파일로 저장되었습니다.
모델이 성공적으로 로드되었습니다.


### 새로운 wav 파일 분류 함수

In [16]:
# def classify_audio(audio_path):
#     try:
#         # 음원 파일 로드
#         y, sr = librosa.load(audio_path, sr=44100)

#         # MFCC 특징 추출
#         mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)

#         # MFCC 평균값 사용
#         mfccs_processed = np.mean(mfccs, axis=1).reshape(1, -1)

#         # 예측 수행
#         predicted_label_encoded = model2.predict(mfccs_processed)[0]
#         predicted_label = label_encoder.inverse_transform([predicted_label_encoded])[0]

#         return predicted_label

#     except Exception as e:
#         print(f"Error processing audio file: {e}")
#         return "Unknown"

# wav 파일 분류 테스트

## 기타소음

### 고양이

In [37]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/동물/15.고양이"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_cat = pd.DataFrame(results)
results_df_cat.to_csv("cat.csv", index=False)
print("Classification completed. Results saved to 'cat.csv'")

100%|██████████| 274/274 [00:19<00:00, 14.02it/s]

Classification completed. Results saved to 'cat.csv'


In [57]:
results_df_cat['Predicted Label'].value_counts()

기타소음      269
차량사이렌       4
이륜차주행음      1
Name: Predicted Label, dtype: int64

In [25]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_cat)  # 전체 파일 개수
etc_noise_count = results_df_cat[results_df_cat['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 274
'기타소음' Count: 269
'기타소음' Ratio: 98.18%


### 항공기

In [22]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/교통소음/3.항공기/6.비행기"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_airplane = pd.DataFrame(results)
results_df_airplane.to_csv("airplane.csv", index=False)
print("Classification completed. Results saved to 'airplane.csv'")

100%|██████████| 191/191 [00:59<00:00,  3.20it/s]

Classification completed. Results saved to 'airplane.csv'


In [58]:
results_df_airplane['Predicted Label'].value_counts()

기타소음      172
이륜차주행음     11
차량주행음       7
차량사이렌       1
Name: Predicted Label, dtype: int64

In [23]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_airplane)  # 전체 파일 개수
etc_noise_count = results_df_airplane[results_df_airplane['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 191
'기타소음' Count: 172
'기타소음' Ratio: 90.05%


### 헬리콥터

In [27]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/교통소음/3.항공기/7.헬리콥터"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_helicopter = pd.DataFrame(results)
results_df_helicopter.to_csv("helicopter.csv", index=False)
print("Classification completed. Results saved to 'helicopter.csv'")

100%|██████████| 435/435 [00:53<00:00,  8.12it/s]

Classification completed. Results saved to 'helicopter.csv'


In [59]:
results_df_helicopter['Predicted Label'].value_counts()

기타소음     434
차량사이렌      1
Name: Predicted Label, dtype: int64

In [28]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_helicopter)  # 전체 파일 개수
etc_noise_count = results_df_helicopter[results_df_helicopter['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 435
'기타소음' Count: 434
'기타소음' Ratio: 99.77%


### 가전 청소기

In [30]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/가전/12.청소기"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_vacuum = pd.DataFrame(results)
results_df_vacuum.to_csv("vacuum.csv", index=False)
print("Classification completed. Results saved to 'vacuum.csv'")

100%|██████████| 102/102 [00:48<00:00,  2.12it/s]

Classification completed. Results saved to 'vacuum.csv'


In [60]:
results_df_vacuum['Predicted Label'].value_counts()

기타소음      97
이륜차주행음     2
차량경적       2
이륜차경적      1
Name: Predicted Label, dtype: int64

In [31]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_vacuum)  # 전체 파일 개수
etc_noise_count = results_df_vacuum[results_df_vacuum['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 102
'기타소음' Count: 97
'기타소음' Ratio: 95.10%


## 교통소음

### 차량 사이렌

In [11]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/1.Car/2.siren_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_car_siren = pd.DataFrame(results)
results_df_car_siren.to_csv("car_siren.csv", index=False)
print("Classification completed. Results saved to 'car_siren.csv'")

100%|██████████| 249/249 [00:31<00:00,  7.91it/s]

Classification completed. Results saved to 'car_siren.csv'


In [12]:
results_df_car_siren['Predicted Label'].value_counts()

차량사이렌     240
차량주행음       4
이륜차주행음      3
기타소음        1
이륜차경적       1
Name: Predicted Label, dtype: int64

In [13]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_car_siren)  # 전체 파일 개수
etc_noise_count = results_df_car_siren[results_df_car_siren['Predicted Label'] == '차량사이렌'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'차량사이렌' Count: {etc_noise_count}")
print(f"'차량사이렌' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 249
'차량사이렌' Count: 240
'차량사이렌' Ratio: 96.39%


### 차량 경적

In [14]:
folder_path = "/home/ubuntu/data/raw_data/1.Car/1.horn_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_car_horn = pd.DataFrame(results)
results_df_car_horn.to_csv("car_horn.csv", index=False)
print("Classification completed. Results saved to 'car_horn.csv'")

100%|██████████| 3189/3189 [05:27<00:00,  9.73it/s]

Classification completed. Results saved to 'car_horn.csv'


In [15]:
results_df_car_horn['Predicted Label'].value_counts()

차량경적      3157
이륜차경적       16
이륜차주행음      11
차량사이렌        2
기타소음         2
차량주행음        1
Name: Predicted Label, dtype: int64

In [16]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_car_horn)  # 전체 파일 개수
etc_noise_count = results_df_car_horn[results_df_car_horn['Predicted Label'] == '차량경적'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'차량경적' Count: {etc_noise_count}")
print(f"'차량경적' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 3189
'차량경적' Count: 3157
'차량경적' Ratio: 99.00%


### 차량 주행음

In [38]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/1.Car/3.driving_sound_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_car_driving = pd.DataFrame(results)
results_df_car_driving.to_csv("car_driving.csv", index=False)
print("Classification completed. Results saved to 'car_driving.csv'")

100%|██████████| 227/227 [00:27<00:00,  8.24it/s]

Classification completed. Results saved to 'car_driving.csv'


In [ ]:
results_df_car_driving['Predicted Label'].value_counts()

In [ ]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_car_driving)  # 전체 파일 개수
etc_noise_count = results_df_car_driving[results_df_car_driving['Predicted Label'] == '차량주행음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'차량주행음' Count: {etc_noise_count}")
print(f"'차량주행음' Ratio: {etc_noise_ratio:.2f}%")

### 이륜차 경적

In [ ]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/4.horn_of_motorcycle"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_motorcycle_horn = pd.DataFrame(results)
results_df_motorcycle_horn.to_csv("motorcycle_horn.csv", index=False)
print("Classification completed. Results saved to 'motorcycle_horn.csv'")

In [ ]:
results_df_motorcycle_horn['Predicted Label'].value_counts()

In [ ]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_motorcycle_horn)  # 전체 파일 개수
etc_noise_count = results_df_motorcycle_horn[results_df_motorcycle_horn['Predicted Label'] == '이륜차경적'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'이륜차경적' Count: {etc_noise_count}")
print(f"'이륜차경적' Ratio: {etc_noise_ratio:.2f}%")

### 이륜차 주행음

In [ ]:
folder_path = "/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/5.driving_sound_of_motorcycle"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df_motorcycle_driving = pd.DataFrame(results)
results_df_motorcycle_driving.to_csv("motorcycle_driving.csv", index=False)
print("Classification completed. Results saved to 'motorcycle_driving.csv'")

In [ ]:
results_df_motorcycle_driving['Predicted Label'].value_counts()

In [ ]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df_motorcycle_driving)  # 전체 파일 개수
etc_noise_count = results_df_motorcycle_driving[results_df_motorcycle_driving['Predicted Label'] == '이륜차주행음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'이륜차주행음' Count: {etc_noise_count}")
print(f"'이륜차주행음' Ratio: {etc_noise_ratio:.2f}%")